텐서플로우 2.0 기반 huggingface 라이브러리
KOBERT를 활용한 네이버쇼핑 감성분석

#목차
<b>1) 네이버 쇼핑 데이터 불러오기 및 전처리  
2) BERT 인풋에 맞게 변환  
3) 버트를 활용한 감성분석 모델 생성  
4) 훈련 및 성능 검증  
5) 실제 데이터 적용</b>

# KOBERT 활용한 네이버쇼핑 감성분석 모델

huggingface 패키지를 Colab에 설치

In [ ]:
!pip install transformers==2.10

In [ ]:
!pip install tensorflow-addons

In [3]:
!pip install -q pyyaml h5py

텐서플로우 2와 필요한 모듈들을 임포트



In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

구글 드라이브와 Colab을 연동하기

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


사용할 네이버 쇼핑 감상분석 데이터를 다운로드

In [ ]:
# 네이버 쇼핑리뷰 감정분석 데이터 다운로드
!git clone https://github.com/bab2min/corpus

In [ ]:
%cd /content/corpus/sentiment
!ls

딥러닝 훈련에 사용 할 train 데이터와 test 데이터를 pandas dataframe 형식으로 불러오기

In [53]:
# 파인튜닝용 쇼핑리뷰데이터
df = pd.read_csv("naver_shopping.txt", sep='\t', names=['label', 'document'])
df.shape

(200000, 2)

In [54]:
# 키워드추출용 특정상품리뷰데이터
df_ = pd.read_csv("/content/gdrive/MyDrive/DataScience/data/navershopping_airpod/review_airpod_encoded.csv")
df_.rename(columns = {'star':'label', 'review':'document'}, inplace = True)
# 평점 3점 행 삭제
idx = df_[df_['label']==3].index
df_ = df_.drop(idx)
df_ = df_.drop(['type', 'category'], axis=1)

In [55]:
# 평점을 [1,2] 점은 부정, [4,5] 점은 긍정으로 분류
df['label'] = df['label'].replace(1, 0)
df['label'] = df['label'].replace(2, 0)
df['label'] = df['label'].replace(4, 1)
df['label'] = df['label'].replace(5, 1)

df_['label'] = df_['label'].replace(1, 0)
df_['label'] = df_['label'].replace(2, 0)
df_['label'] = df_['label'].replace(4, 1)
df_['label'] = df_['label'].replace(5, 1)

In [56]:
df

,label,document
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,0,장마라그런가!!! 달지않아요
199996,1,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,1,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,1,넘이쁘고 쎄련되보이네요~


In [57]:
df['label'].value_counts(normalize=True)

0    0.500185
1    0.499815
Name: label, dtype: float64

In [58]:
df_

,document,label
0,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,1
1,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",1
2,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,1
3,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,1
4,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,1
...,...,...
3679,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,1
3680,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,1
3681,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",1
3682,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,1


In [59]:
# 타겟레이블 분포 확인 타겟레이블 분포 확인
df_['label'].value_counts(normalize=True)
df_['label'].value_counts(normalize=True)

1    0.996672
0    0.003328
Name: label, dtype: float64

In [60]:
# 훈련셋과 검증셋으로 분리
train, test = train_test_split(df,
                               random_state=3, 
                               test_size=0.25,
                               stratify=df['label'])

print(f"학습용데이터 : {train.shape}\n테스트용데이터 : {test.shape}")

학습용데이터 : (150000, 2)
테스트용데이터 : (50000, 2)


In [61]:
# 데이터별 타겟 분포 확인
print(train['label'].value_counts(normalize=True))
print(test['label'].value_counts(normalize=True))

0    0.500187
1    0.499813
Name: label, dtype: float64
0    0.50018
1    0.49982
Name: label, dtype: float64


In [62]:
# 훈련셋의 앞부분 출력
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
df_.reset_index(drop=True, inplace=True)
train.head(10)

,label,document
0,0,사용하기불편해요 잘녹지도않고
1,1,사이즈 딱좋아요 휴대성도 좋구요~^^
2,0,오래된 가족 쇼파를 산뜻하게 변화시켜준 것 좋았는데 안고 나면 자꾸 앉은 자리 커버...
3,0,너무약해요 복근은동으로는 좀 ..
4,1,튼튼하고 사이즈도 만족스러워요
5,0,"오프라인보다 저렴한 가격, 빠른 배송은 아주 만족합니다. 하지만 포장은 뜯으니 일부..."
6,0,재구매 첫번째 사진은 비교하기 위해 올려놓았습니다. 두번째사진은 작은 사이즈라 전부...
7,1,깔끔하니 아주이쁘네요~~
8,0,재구매 세번째 구매인데 ㅠㅠ이번엔 상태가 좀 안좋은거 같아요 너무 오래 후숙된 느낌...
9,1,구매가 편해서 좋네요


## 버트 인풋 만들기

한국어 데이터로 훈련되었고, SKT에서 만든 KoBERT를 사용  

https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드 가져옴

In [ ]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

kobert 토크나이즈를 임포트

In [63]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌  
tokenizer.tokenize => 문장을 토큰화

In [64]:
print(tokenizer.encode("너무 맛있어서 다시 주문했어요"))

[2, 1458, 1967, 7141, 6855, 6553, 1574, 4227, 7874, 3]


In [65]:
print(tokenizer.tokenize("너무 맛있어서 다시 주문했어요"))

['▁너무', '▁맛', '있', '어', '서', '▁다시', '▁주문', '했어요']


네이버 쇼핑 리뷰 문장들을 버트 인풋으로 변환  
문장은 토큰 인풋, 세그먼트 인풋, 마스크 인풋으로 변환  
huggingface에서의 순서 [토큰 인풋, 마스크 인풋, 세그먼트 인풋]

In [ ]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 64
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

In [ ]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

## 버트를 활용한 감성분석 모델 만들기

In [29]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [30]:
bert_outputs

(<KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'tf_bert_model')>,
 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)

In [31]:
bert_outputs = bert_outputs[1]

In [32]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

In [33]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [34]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 64, 768), (N 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]            

## 훈련 및 성능 검증

모델 학습

In [ ]:
sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=64, validation_data=(test_x, test_y))

모델 저장

In [ ]:
# sentiment_model.save('/content/gdrive/MyDrive/DataScience/model/mymodel')

In [ ]:
# sentiment_model.save_weights('/content/gdrive/MyDrive/DataScience/model/saveweights/save.h5')

In [36]:
sentiment_model.load_weights('/content/gdrive/MyDrive/DataScience/model/saveweights/save.h5')

훈련 모델의 예측 성능을 F1 SCORE로 체크하기 위한 작업

In [37]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [38]:
test_set = predict_load_data(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 50000/50000 [00:06<00:00, 7703.86it/s]


In [39]:
test_set

[array([[   2, 3647, 5377, ...,    1,    1,    1],
        [   2, 3093,  463, ...,    1,    1,    1],
        [   2, 2779, 7836, ...,    1,    1,    1],
        ...,
        [   2, 1185, 7199, ...,    1,    1,    1],
        [   2, 1458,  517, ...,    1,    1,    1],
        [   2, 4379, 7096, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [40]:
preds = sentiment_model.predict(test_set)

In [41]:
# 부정이면 0, 긍정이면 1 출력
preds

array([[0.00332655],
       [0.00229315],
       [0.9933577 ],
       ...,
       [0.00197776],
       [0.00173869],
       [0.00149035]], dtype=float32)

우리가 훈련한 모델을 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [42]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94     25009
           1       0.93      0.95      0.94     24991

    accuracy                           0.94     50000
   macro avg       0.94      0.94      0.94     50000
weighted avg       0.94      0.94      0.94     50000



In [43]:
import logging
tf.get_logger().setLevel(logging.ERROR)

# 실제 데이터 적용

In [44]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value)

In [45]:
review_evaluation_predict("두번 다시 안 살꺼에요")

(부정 확률 : 0.99) 부정적인 리뷰입니다.


In [46]:
review_evaluation_predict("배송이 좀 느리긴 했어도 품질 자체는 믿을만해요")

(긍정 확률 : 0.98) 긍정적인 리뷰입니다.


In [47]:
review_evaluation_predict("디자인이 매우 세련되었으나 쉽게 망가지네요. 내구도가 엉망이에요")

(부정 확률 : 1.00) 부정적인 리뷰입니다.


### 에어팟 리뷰데이터 분류

In [48]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      return "(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value)
    elif predict_answer == 1:
      return "(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value

In [49]:
df_['evaluation'] = df_['document'].apply(review_evaluation_predict)

In [50]:
df_

,document,label,evaluation
0,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
1,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
2,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
4,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
...,...,...,...
3601,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3602,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3603,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",1,(긍정 확률 : 0.86) 긍정적인 리뷰입니다.
3604,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,1,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.


In [51]:
df_['document'][0]

'2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_<'

In [52]:
# df_.to_csv('/content/gdrive/MyDrive/DataScience/data/navershopping_airpod/sentimental_output.csv', encoding='utf-8-sig', mode='w')